In [ ]:
#!rm /kaggle/working/*

!wget https://github.com/meituan/YOLOv6/archive/refs/tags/0.2.1.zip
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.1/yolov6m_base.pt

In [ ]:
!unzip /kaggle/working/0.2.1.zip -d /kaggle/working/

In [ ]:
%cd /kaggle/working/YOLOv6-0.2.1/
!pip install -qr requirements.txt
!pip install gdown
%cd /kaggle/working/

In [ ]:
!unzip /kaggle/working/BNVD.zip -d /kaggle/working/BNVD/

In [7]:
data_path = "/kaggle/working/BNVD/data.yaml"

In [8]:
%%writefile {data_path}

train: /kaggle/working/BNVD/images/train
val: /kaggle/working/BNVD/images/valid
test: /kaggle/working/BNVD/images/test

nc: 17
names: ["Bicycle","Bus","Bhotbhoti","Car","CNG","Easybike","Leguna","Motorbike","MPV","Pedestrian","Pickup","PowerTiller","Rickshaw","ShoppingVan","Truck","Van","Wheelbarrow"]

Overwriting /kaggle/working/ODS/data.yaml


In [11]:
config_path = "/kaggle/working/YOLOv6-0.2.1/configs/yolov6m_finetune.py"

In [ ]:
%%writefile {config_path}
# YOLOv6m model
model = dict(
    type='YOLOv6m',
    pretrained='/kaggle/working/yolov6m_base.pt',
    depth_multiple=0.60,
    width_multiple=0.75,
    backbone=dict(
        type='CSPBepBackbone',
        num_repeats=[1, 6, 12, 18, 6],
        out_channels=[64, 128, 256, 512, 1024],
        csp_e=float(2)/3,
        ),
    neck=dict(
        type='CSPRepPANNeck',
        num_repeats=[12, 12, 12, 12],
        out_channels=[256, 128, 128, 256, 256, 512],
        csp_e=float(2)/3,
        ),
    head=dict(
        type='EffiDeHead',
        in_channels=[128, 256, 512],
        num_layers=3,
        begin_indices=24,
        anchors=1,
        out_indices=[17, 20, 23],
        strides=[8, 16, 32],
        iou_type='giou',
        use_dfl=True,
        reg_max=16, #if use_dfl is False, please set reg_max to 0
        distill_weight={
            'class': 1.0,
            'dfl': 1.0,
        },
    )
)
solver = dict(
    optim='SGD',
    lr_scheduler='Cosine',
    lr0=0.0032,
    lrf=0.12,
    momentum=0.843,
    weight_decay=0.00036,
    warmup_epochs=2.0,
    warmup_momentum=0.5,
    warmup_bias_lr=0.05
)
data_aug = dict(
    hsv_h=0.0138,
    hsv_s=0.664,
    hsv_v=0.464,
    degrees=0.373,
    translate=0.245,
    scale=0.898,
    shear=0.602,
    flipud=0.00856,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.243,
)

In [ ]:
%cd /kaggle/working/YOLOv6-0.2.1/
!python tools/train.py --batch-size 64 --epochs 100 --conf-file configs/yolov6m_finetune.py  --data-path /kaggle/working/ODS/data.yaml --img-size 640 --device 0,1